In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [3]:
def getClickCard(soup, job_category, url_obj):
    
    # get all the clickcard data on the first search page
    clickcard = soup.find_all('div', class_=['row', 'result'])
    
    # save job_category as a tempory value
    job_category_temp = job_category
    
    # change the all " " spaces of job_category to "+" e.g "data engineer" -> "data+engineer"
    job_category = job_category.replace("+", " ")

    # iterate the clickcard list lengh
    for i in range(len(clickcard)):
        
        # get the first record of job title -> <a href="xxxx" title="xxxxxx">...</a>
        jobTitle = clickcard[i].find('a').get('title')

        # get the first record of url link -> <a href="xxxx">...</a>
        links = clickcard[i].find('a').get('href')
        
        # combine "https://www.indeed.hk" + /rc/xxxxxxxxxx/xxxxx/xxx
        full_url = "https://www.indeed.hk" + links
        
        vj_link = full_url[32:48]
        
        if(vj_link[0:1] != "?"):
            full_url = url_obj + "&vjk=" + vj_link
            print(full_url)
        else:
            full_url = np.nan
    
        # get the company name
        company_temp = clickcard[i].find(class_='company').get_text()
        company_name = company_temp.lstrip()
        #company_temp = company_temp.split() # trim to a list e.g ['Gold', 'Sachs']
        #company_name = " ".join(company_temp) # join the list together with " " spaces

        # get the location
        location = clickcard[i].find(class_='location').get_text()

        # get the summery
        summery = clickcard[i].find(class_='summary').get_text().lstrip()
        
        # try if the ""date = clickcard[i].find(class_='date').get_text()"" is work, if not, go to np.nan
        try:
            # get date
            date = clickcard[i].find(class_='date').get_text()
        except:
            date = np.nan
        
        #global df2 is share the both df2 outside the function
        global df2
        df2 = df2.append({'Job Category':job_category,'Job Title':jobTitle, 'Company Name':company_name, 'Location':location,'Summary':summery,'Date':date,'URL':full_url}, ignore_index=True)

In [4]:
search_page_list = "https://www.indeed.hk/jobs?q="
keywords = ["data engineer", "senior data engineer"]

keywords_process = []
for i in keywords:
    keywords_process.append(i.replace(" ", "+"))
print(keywords_process)

pattern_dq = "%22" # '"' double quote
pattern_as_list = "&sort=date"

#x = 0
df2 = pd.DataFrame(columns=['Job Category','Job Title', 'Company Name', 'Location', 'Summary', 'Date', 'URL'])

for i in keywords_process:
    record_num = 0
    while True:
        url_pattern = search_page_list + pattern_dq + i + pattern_dq + pattern_as_list + "&start=" + str(record_num)
        print(url_pattern)
        page = requests.get(url_pattern)
        soup = BeautifulSoup(page.content, 'html.parser')
        getClickCard(soup, i, page.url)
        postYourResume_temp = soup.find_all('div')
        postYourResume_data = "".join(str(postYourResume_temp))

        if "Post your resume" in postYourResume_data:
            break
        else:
            record_num += 10
        #x += 1

['data+engineer', 'senior+data+engineer']
https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0
https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=a5e70419e8d1a004
https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=2f458409f433cf1e
https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=5da569629c8b4c69
https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=a1e339764d9e6f2d
https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=cefe9816f5e25714
https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=af25aa05f0edbc76
https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=1bb8ce1f79634b42
https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=41a6d8701d3457f4
https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=24074c092c848a4b
https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=4be76e166a9e598a
https://www

In [5]:
new_df = df2
new_df = new_df[new_df.URL.notnull()]
new_df

,Job Category,Job Title,Company Name,Location,Summary,Date,URL
0,data engineer,Equities AI Full Stack Developer,UBS,Hong Kong,Are you a talented data engineer that wants to...,Today,https://www.indeed.hk/jobs?q=%22data+engineer%...
1,data engineer,Edge Data Engineering - Data Engineer,Goldman Sachs,Hong Kong,Engineer streaming data processing pipelines. ...,3 days ago,https://www.indeed.hk/jobs?q=%22data+engineer%...
2,data engineer,Market Data Engineer / Data Analyst,BAH Partners,Hong Kong,"This role has many exciting facets to it, incl...",3 days ago,https://www.indeed.hk/jobs?q=%22data+engineer%...
3,data engineer,Data Engineer - Fintech Firm,Hays,Hong Kong,5+ years of Data Engineering experience. Deliv...,6 days ago,https://www.indeed.hk/jobs?q=%22data+engineer%...
4,data engineer,Market Data Engineer,BAH Partners,Hong Kong,My client is a leading investment bank who is ...,13 days ago,https://www.indeed.hk/jobs?q=%22data+engineer%...
5,data engineer,"Research Data Scientist /Big Data Engineer,AI ...",SmartBlkTrade,Hong Kong,"Handle the tasks of data analytics, big data, ...",7 days ago,https://www.indeed.hk/jobs?q=%22data+engineer%...
6,data engineer,"Senior Data Engineer - HSBC Operations, Servic...",HSBC,"Kowloon City, Kowloon",We are currently seeking a high caliber profes...,7 days ago,https://www.indeed.hk/jobs?q=%22data+engineer%...
7,data engineer,Svc Info Developer I,Fruition Partners,Hong Kong,Interacts with the Client and project roles (e...,10 days ago,https://www.indeed.hk/jobs?q=%22data+engineer%...
8,data engineer,Data - Speech Analytics,Robert Walters,Hong Kong Island,"For Data Engineer, they should possess good sp...",9 days ago,https://www.indeed.hk/jobs?q=%22data+engineer%...
9,data engineer,Data Engineer,Code Pilot,Hong Kong,Code Pilot is replacing the crusty old resume ...,14 days ago,https://www.indeed.hk/jobs?q=%22data+engineer%...


In [8]:
new_df_url = new_df['URL']
new_df_url

0     https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=a5e70419e8d1a004       
1     https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=2f458409f433cf1e       
2     https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=5da569629c8b4c69       
3     https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=a1e339764d9e6f2d       
4     https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=cefe9816f5e25714       
5     https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=af25aa05f0edbc76       
6     https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=1bb8ce1f79634b42       
7     https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=41a6d8701d3457f4       
8     https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=24074c092c848a4b       
9     https://www.indeed.hk/jobs?q=%22data+engineer%22&sort=date&start=0&vjk=4be76e166a9e59

In [7]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
new_df.to_csv("data_engineer.csv")

In [ ]:
temp_df1 = pd.read_csv('indeedjobs.csv', index_col=0)
temp_df2 = pd.read_csv('Business Intelligence.csv', index_col=0)
temp_df3 = pd.read_csv('data_engineer.csv', index_col=0)
temp_df4 = pd.read_csv('Data Analyst.csv', index_col=0)
temp_df5 = pd.read_csv('Business Intelligence Manager.csv', index_col=0)
temp_df6 = pd.read_csv('Business Intelligence Director.csv', index_col=0)
print(temp_df1.shape, temp_df2.shape, temp_df3.shape, temp_df4.shape, temp_df5.shape, temp_df6.shape)

In [ ]:
temp_df1 = temp_df1.rename(columns={'Job_Category':'Job Category', 'Job_Title': 'Job Title', 'Company_Name':'Company Name'})

In [ ]:
total_df = pd.concat([temp_df1, temp_df2, temp_df3, temp_df4, temp_df5, temp_df6], ignore_index=True)
total_df